# Desription
Bike sharing systems are a means of renting bicycles where the process of obtaining membership, rental, and bike return is automated via a network of kiosk locations throughout a city. Using these systems, people are able rent a bike from a one location and return it to a different place on an as-needed basis. Currently, there are over 500 bike-sharing programs around the world.

The data generated by these systems makes them attractive for researchers because the duration of travel, departure location, arrival location, and time elapsed is explicitly recorded. Bike sharing systems therefore function as a sensor network, which can be used for studying mobility in a city. In this competition, participants are asked to combine historical usage patterns with weather data in order to forecast bike rental demand in the Capital Bikeshare program in Washington, D.C.

# Data
You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period.

Data Fields
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals


Data Fields
1. datetime - hourly date + timestamp  
* season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
* weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
	* 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
	* 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
	* 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* count - number of total rentals

In [69]:
import pandas as pd
import numpy as np

diro = 'C:\\Users\\Lenovo\\PycharmProjects\\Kaggle\\Project6_BikeDemand\\data\\'
train = pd.read_csv(diro + 'train.csv', parse_dates=True)
test = pd.read_csv(diro + 'test.csv')
samsub = pd.read_csv(diro + 'sampleSubmission.csv')

print('Import completed')

Import completed


In [70]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [10]:
print('Number of rows \n\ttrain:{}\n\ttest:{}\nNumber of columns\n\ttrain:{}\n\ttest:{}'
      .format(train.shape[0], test.shape[0], train.shape[1], test.shape[1]))

Number of rows 
	train:10886
	test:6493
Number of columns
	train:12
	test:9


In [32]:
# common columns
common = list(set(train.columns).intersection(set(test.columns)))

In [57]:
nans = pd.concat([train.isnull().sum(), test.isnull().sum()], axis=1)
nans.columns=['Train', 'Test']
print('Amount of Null values:\n')
print(nans.T)
print('\nColumns which are not in Test data: {}'.format(list(nans[nans['Test'] != 0].index)))

Amount of Null values:

       atemp  casual  count  datetime  holiday  humidity  registered  season  \
Train    0.0     0.0    0.0       0.0      0.0       0.0         0.0     0.0   
Test     0.0     NaN    NaN       0.0      0.0       0.0         NaN     0.0   

       temp  weather  windspeed  workingday  
Train   0.0      0.0        0.0         0.0  
Test    0.0      0.0        0.0         0.0  

Columns which are not in Test data: ['casual', 'count', 'registered']


In [64]:
train.loc[:, common].describe().loc[['mean', 'min', 'max'], :]

,temp,atemp,windspeed,weather,humidity,season,holiday,workingday
mean,20.23086,23.655084,12.799395,1.418427,61.88646,2.506614,0.028569,0.680875
min,0.82000,0.760000,0.000000,1.000000,0.00000,1.000000,0.000000,0.000000
max,41.00000,45.455000,56.996900,4.000000,100.00000,4.000000,1.000000,1.000000


In [68]:
train.datetime

0        2011-01-01 00:00:00
1        2011-01-01 01:00:00
2        2011-01-01 02:00:00
3        2011-01-01 03:00:00
4        2011-01-01 04:00:00
5        2011-01-01 05:00:00
6        2011-01-01 06:00:00
7        2011-01-01 07:00:00
8        2011-01-01 08:00:00
9        2011-01-01 09:00:00
10       2011-01-01 10:00:00
11       2011-01-01 11:00:00
12       2011-01-01 12:00:00
13       2011-01-01 13:00:00
14       2011-01-01 14:00:00
15       2011-01-01 15:00:00
16       2011-01-01 16:00:00
17       2011-01-01 17:00:00
18       2011-01-01 18:00:00
19       2011-01-01 19:00:00
20       2011-01-01 20:00:00
21       2011-01-01 21:00:00
22       2011-01-01 22:00:00
23       2011-01-01 23:00:00
24       2011-01-02 00:00:00
25       2011-01-02 01:00:00
26       2011-01-02 02:00:00
27       2011-01-02 03:00:00
28       2011-01-02 04:00:00
29       2011-01-02 06:00:00
                ...         
10856    2012-12-18 18:00:00
10857    2012-12-18 19:00:00
10858    2012-12-18 20:00:00
10859    2012-

## Prepara Data

#### Use only columns which occur in test set. 

In [ ]:
train = 